# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [150]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [151]:
# Read your api key environment variable
load_dotenv()
news_api_key = os.getenv("NEWS_API_KEY")

In [152]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=news_api_key)


In [153]:
# Define timeframe for grabbing articles
today = pd.Timestamp.now()
previous = today - pd.Timedelta(4, unit="W")
TIME_FORMAT_STRING = "%Y-%m-%d"


In [154]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_top_headlines(q="bitcoin", category="business", language="en", country="us")

# Print total and show sample
print(f"Total articles about bitcoin: {bitcoin_headlines['totalResults']}")
bitcoin_headlines["articles"][0]


Total articles about bitcoin: 1


{'source': {'id': None, 'name': 'CryptoPotato'},
 'author': 'Jordan Lyanchev',
 'title': 'Bitcoin Soars $1700 In 48-Hours: Trading Over $19K As Market Cap Gains $20B (Market Watch) - CryptoPotato',
 'description': 'The crypto market cap eyes $560 billion as bitcoin reclaimed $19K once again. Altcoins? Most lack behind due to the rising BTC dominance',
 'url': 'https://cryptopotato.com/bitcoin-soars-1700-in-48-hours-trading-over-19k-as-market-cap-gains-20b-market-watch/',
 'urlToImage': 'https://cryptopotato.com/wp-content/uploads/2020/11/BitcoinRocket-min-1.jpg',
 'publishedAt': '2020-12-13T08:06:45Z',
 'content': 'Just two days after bottoming at $17,500, bitcoin has recovered the losses and spiked back beyond $19K, as of writing these lines. Most major altcoins are also green, besides Ripple (XRP), which cont… [+3136 chars]'}

In [156]:
# Fetch the Ethereum news articles
etherium_headlines = newsapi.get_everything(q='etherium',
                                      from_param=previous.strftime(TIME_FORMAT_STRING),
                                      to=today.strftime(TIME_FORMAT_STRING),
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# Print total and show sample
# print(f"Total articles about etherium: {etherium_headlines['totalResults']}")
# etherium_headlines["articles"][0]


In [157]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_df = pd.DataFrame.from_dict(bitcoin_headlines["articles"])
bitcoin_df.head()


,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'CryptoPotato'}",Jordan Lyanchev,Bitcoin Soars $1700 In 48-Hours: Trading Over ...,The crypto market cap eyes $560 billion as bit...,https://cryptopotato.com/bitcoin-soars-1700-in...,https://cryptopotato.com/wp-content/uploads/20...,2020-12-13T08:06:45Z,"Just two days after bottoming at $17,500, bitc..."


In [158]:
# Create the ethereum sentiment scores DataFrame
etherium_df = pd.DataFrame.from_dict(etherium_headlines["articles"])
etherium_df.head()


""


In [81]:
# Describe the Bitcoin Sentiment



In [82]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [83]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [84]:
# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [85]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words

    # Convert the words to lowercase
    
    # Remove the punctuation
    
    # Remove the stop words
    
    # Lemmatize Words into root words
    
    return tokens


In [86]:
# Create a new tokens column for bitcoin
# YOUR CODE HERE!

In [87]:
# Create a new tokens column for ethereum
# YOUR CODE HERE!

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [88]:
from collections import Counter
from nltk import ngrams

In [89]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [90]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [91]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [92]:
# Get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [93]:
# Get the top 10 words for Ethereum
# YOUR CODE HERE!

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [94]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [95]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [96]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [97]:
import spacy
from spacy import displacy

In [98]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [99]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [100]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

In [101]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [102]:
# Render the visualization
# YOUR CODE HERE!

In [103]:
# List all Entities
# YOUR CODE HERE!

---

## Ethereum NER

In [104]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

In [105]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [106]:
# Render the visualization
# YOUR CODE HERE!

In [107]:
# List all Entities
# YOUR CODE HERE!